In [3]:
%pip install fireworks-ai python-dotenv

  Obtaining dependency information for fireworks-ai from https://files.pythonhosted.org/packages/7a/a1/167bcc97387dd2b1c3e27dc141c6388ce2b194b28a429c4a3eb34aa15b3a/fireworks_ai-0.17.16-py3-none-any.whl.metadata
  Obtaining dependency information for httpx-ws from https://files.pythonhosted.org/packages/03/3d/2113a5c7af9a13663fa026882d0302ed4142960388536f885dacd6be7038/httpx_ws-0.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for httpx-sse from https://files.pythonhosted.org/packages/e1/9b/a181f281f65d776426002f330c31849b86b31fc9d848db62e16f03ff739f/httpx_sse-0.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for mmh3>=4.1.0 from https://files.pythonhosted.org/packages/12/dd/7cbc30153b73f08eeac43804c1dbc770538a01979b4094edbe1a4b8eb551/mmh3-5.1.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for betterproto-fw[compiler]>=2.0.3 from https://files.pythonhosted.org/packages/0c/85/fdd477aa981bf80ff610bc256d3c5b079a4294cab74ff6e13cf832

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'c:\\Python312\\Scripts\\ruff.exe'
Consider using the `--user` option or check the permissions.



In [3]:
import os
from fireworks.client import Fireworks
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

API_KEY = os.environ.get("FIREWORKS_API_KEY")
API_MODEL = os.environ.get("FIREWORKS_MODEL")
fw = Fireworks(api_key=API_KEY)  # Fixed variable name (was api_key)
model_id = API_MODEL

conceptos = [
    "inflación",
    "nutrición básica",
    "cómo llenar un cheque",
    "ecuaciones diferenciales",
    "Shakespeare",
    "libros",
    "machine learning",
    "formatos estáticos",
    "mecánica cuántica"
]

prompt = (
    "Dada la siguiente lista de conceptos, responde solo en JSON clasificándolos en conocidos "
    "y no conocidos por una persona sin estudios universitarios.\n\n" +
    "\n".join(f"- {c}" for c in conceptos) +
    "\n\nResponde estrictamente en el siguiente formato JSON:\n\n" +
    '{\n  "conocidos": [...],\n  "no_conocidos": [...]\n}'
)

response = fw.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "user", "content": prompt}  # Fixed: Using the prompt you constructed
    ]
)

print(response.choices[0].message.content.strip())


KeyboardInterrupt: 

In [2]:
import re
import spacy
from spacy.matcher import Matcher

# Load the Spanish model from spaCy
nlp = spacy.load("es_core_news_sm")  # medium model works better than 'sm' for this

# Sample full document
text = (
    """Las estrategias automatizadas, por otro lado, no tienen patrones
comunes claros , suelen ser un composición de técnicas como: Segmentación de
audio, ya sea a través de detección de silencios u otros features como el tono,
la energía o sentimientos. Pero aún es difícil obtener resultados fiables  cuando 
se trata de diferentes acentos, entornos ruidosos y jerga de dominio específico. El potencial mostrado por el aprendizaje
profundo propició el estudio de enfoques exitosos para entrenar pasos de modelado que antes eran separados."""
)

# Split text into paragraphs
paragraphs = re.split(r"\n\n+", text)
paragraph_starts = []
cursor = 0
for para in paragraphs:
    paragraph_starts.append(cursor)
    cursor += len(para) + 2  # account for \n\n
# Process the text
doc = nlp(text)

# Define a matcher to extract potential concepts (noun phrases and named entities)
matcher = Matcher(nlp.vocab)

# Pattern for noun phrases (a noun possibly preceded by adjectives/determiners and followed by modifiers)
pattern = [
    {"POS": "DET", "OP": "?"},
    {"POS": "ADJ", "OP": "*"},
    {"POS": "NOUN", "OP": "+"},
    {"POS": "ADP", "OP": "*"},
    {"POS": "DET", "OP": "*"},
    {"POS": "ADJ", "OP": "*"},
    {"POS": "NOUN", "OP": "*"}
]
matcher.add("ConceptPattern", [pattern])

# Apply the matcher
matches = matcher(doc)

def filter_spans(spans):
    result = []
    for match in spans:
        if not any(
            (other.start <= match.start and other.end >= match.end)
            and (other != match)  # Avoid comparing the same element
            for other in spans
        ):
            result.append(match)
    return result

# Extract matched phrases, keeping only longest overlapping ones
matched_spans = [doc[start:end] for _, start, end in matches]
matched_spans = filter_spans(matched_spans)  # keeps only the longest overlapping matches

# Build set of unique concepts with positions
concepts = set()
concept_spans = []
for span in matched_spans:
    span_text = span.text.strip()
    if len(span_text.split()) > 1 and not all(token.pos_ == "DET" for token in span):
        if span_text not in concepts:
            concepts.add(span_text)
            concept_spans.append((span_text, span.start_char, span.end_char))

# Also add named entities
for ent in doc.ents:
    if ent.label_ in ["PER", "ORG", "LOC", "MISC"] and ent.text.strip() not in concepts:
        concepts.add(ent.text.strip())
        concept_spans.append((ent.text.strip(), ent.start_char, ent.end_char))

# Function to find paragraph index from char position
def get_paragraph_index(start_char):
    for i, para_start in enumerate(paragraph_starts):
        if start_char < para_start:
            return max(0, i - 1)
    return len(paragraph_starts) - 1

# Definition-like keywords
definition_patterns = [
    r"\\b(se refiere a|es|significa|consiste en|denominamos|se conoce como|se define como)\\b"
]
conceptos = []
for c in concept_spans:
    conceptos.append(c[0])

prompt = (
    """Tienes el siguiente texto en español. Tu tarea consiste en:

1. Identificar todos los conceptos, ideas o entidades que aparecen en el primer párrafo y que podrían necesitar una definición. Estos suelen ser frases nominales como "el problema de los acertijos risueños" o "los formatos estáticos".

2. Para cada concepto, revisa los párrafos siguientes y e intenta responder a las siguientes preguntas solo con el contenido del texto: qué es <concepto>?, <concepto> está justificado?. Si las puedes responder, indica:

   - Respuesta por pregunta.

Si un concepto no está definido explícitamente, simplemente indícalo como "No definido".

### Texto:
Las estrategias automatizadas, por otro lado, no tienen patrones
comunes claros , suelen ser un composición de técnicas como: Segmentación de
audio, ya sea a través de detección de silencios u otros features como el tono,
la energía o sentimientos. Pero aún es difícil obtener resultados fiables  cuando 
se trata de diferentes acentos, entornos ruidosos y jerga de dominio específico. El potencial mostrado por el aprendizaje
profundo propició el estudio de enfoques exitosos para entrenar pasos de modelado que antes eran separados.


### Formato de respuesta:

1. Concepto: [texto]
   - Definido: Sí / No
   - Párrafo(s): [número(s) o cita(s)]
   - Respuesta a preguntas: respuesta 1, respuesta 2
    """
)
# response = fw.chat.completions.create(
#     model=model_id,
#     messages=[
#         {"role": "user", "content": prompt}  # Fixed: Using the prompt you constructed
#     ]
# )



# print(response.choices[0].message.content.strip())
# text = (
#     """Las estrategias automatizadas, por otro lado, no tienen patrones
# comunes claros , suelen ser un composición de técnicas como: Segmentación de
# audio, ya sea a través de detección de silencios u otros features como el tono,
# la energía o sentimientos. Pero aún es difícil obtener resultados fiables  cuando 
# se trata de diferentes acentos, entornos ruidosos y jerga de dominio específico. El potencial mostrado por el aprendizaje
# profundo propició el estudio de enfoques exitosos para entrenar pasos de modelado que antes eran separados."""
# )

In [ ]:
import re
import spacy
from spacy.matcher import Matcher

# Load the Spanish model from spaCy
nlp = spacy.load("es_core_news_sm")  # medium model works better than 'sm' for this

# Sample full document
text = (
    """Las estrategias automatizadas, por otro lado, no tienen patrones
comunes claros , suelen ser un composición de técnicas como: Segmentación de
audio, ya sea a través de detección de silencios u otros features como el tono,
la energía o sentimientos. Pero aún es difícil obtener resultados fiables  cuando 
se trata de diferentes acentos, entornos ruidosos y jerga de dominio específico. El potencial mostrado por el aprendizaje
profundo propició el estudio de enfoques exitosos para entrenar pasos de modelado que antes eran separados."""
)

# Split text into paragraphs
paragraphs = re.split(r"\n\n+", text)
paragraph_starts = []
cursor = 0
for para in paragraphs:
    paragraph_starts.append(cursor)
    cursor += len(para) + 2  # account for \n\n
# Process the text
doc = nlp(text)

# Define a matcher to extract potential concepts (noun phrases and named entities)
matcher = Matcher(nlp.vocab)

# Pattern for noun phrases (a noun possibly preceded by adjectives/determiners and followed by modifiers)
pattern = [
    {"POS": "DET", "OP": "?"},
    {"POS": "ADJ", "OP": "*"},
    {"POS": "NOUN", "OP": "+"},
    {"POS": "ADP", "OP": "*"},
    {"POS": "DET", "OP": "*"},
    {"POS": "ADJ", "OP": "*"},
    {"POS": "NOUN", "OP": "*"}
]
matcher.add("ConceptPattern", [pattern])

# Apply the matcher
matches = matcher(doc)

def filter_spans(spans):
    result = []
    for match in spans:
        if not any(
            (other.start <= match.start and other.end >= match.end)
            and (other != match)  # Avoid comparing the same element
            for other in spans
        ):
            result.append(match)
    return result

# Extract matched phrases, keeping only longest overlapping ones
matched_spans = [doc[start:end] for _, start, end in matches]
matched_spans = filter_spans(matched_spans)  # keeps only the longest overlapping matches

# Build set of unique concepts with positions
concepts = set()
concept_spans = []
for span in matched_spans:
    span_text = span.text.strip()
    if len(span_text.split()) > 1 and not all(token.pos_ == "DET" for token in span):
        if span_text not in concepts:
            concepts.add(span_text)
            concept_spans.append((span_text, span.start_char, span.end_char))

# Also add named entities
for ent in doc.ents:
    if ent.label_ in ["PER", "ORG", "LOC", "MISC"] and ent.text.strip() not in concepts:
        concepts.add(ent.text.strip())
        concept_spans.append((ent.text.strip(), ent.start_char, ent.end_char))

# Function to find paragraph index from char position
def get_paragraph_index(start_char):
    for i, para_start in enumerate(paragraph_starts):
        if start_char < para_start:
            return max(0, i - 1)
    return len(paragraph_starts) - 1

# Definition-like keywords
definition_patterns = [
    r"\\b(se refiere a|es|significa|consiste en|denominamos|se conoce como|se define como)\\b"
]
conceptos = []
for c in concept_spans:
    conceptos.append(c[0])

prompt = (
    """Tienes el siguiente capítulo de una tesis en español. Tu tarea consiste en:

1. Identificar las ideas que queden ambiguas o los conceptos que haga falta explicar porque dependan del contexto y no se hayan aclarado en los preliminares, por ejemplo, los adverbios de modo en ocasiones dependen del punto de referencia.
Para esto ten en cuenta que en los preliminares de esta tesis se explicó claramente que: Las palabras de comadreja se pueden identificar con un modelo sería necesario entrenar un corpus anotado en español, con el cual no se cuenta actualmente.

### Texto:
En este trabajo se va a señalar palabras de comadreja. Para esto fue necesario usar una lista porque no se cuenta con un corpus para entrenar algún modelo. Las estrategias automatizadas, por otro lado, no tienen patrones
comunes claros , suelen ser un composición de técnicas como: Segmentación de
audio, ya sea a través de detección de silencios u otros features como el tono,
la energía o sentimientos. Pero aún es difícil obtener resultados fiables  cuando 
se trata de diferentes acentos, entornos ruidosos y jerga de dominio específico. El potencial mostrado por el aprendizaje
profundo propició el estudio de enfoques exitosos para entrenar pasos de modelado que antes eran separados.


### Formato de respuesta:

1. Idea: [texto]
   - Por qué es ambigua: respuesta
   - Párrafo(s): [número(s) o cita(s)]
    """
)
response = fw.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "user", "content": prompt}  # Fixed: Using the prompt you constructed
    ]
)



print(response.choices[0].message.content.strip())
text = (
    """En este trabajo se va a señalar palabras de comadreja. Para esto fue necesario usar una lista porque no se cuenta con un corpus para entrenar algún modelo.Las estrategias automatizadas, por otro lado, no tienen patrones
comunes claros , suelen ser un composición de técnicas como: Segmentación de
audio, ya sea a través de detección de silencios u otros features como el tono,
la energía o sentimientos. Pero aún es difícil obtener resultados fiables  cuando 
se trata de diferentes acentos, entornos ruidosos y jerga de dominio específico. El potencial mostrado por el aprendizaje
profundo propició el estudio de enfoques exitosos para entrenar pasos de modelado que antes eran separados."""
)

1. Idea: Las estrategias automatizadas no tienen patrones comunes claros.
   - Por qué es ambigua: Se refiere a que las estrategias automatizadas no tienen patrones claros, pero no se especifica qué se entiende exactamente por "patrones claros" en este contexto.
   - Párrafo(s): Se encuentra en el segundo párrafo del texto.

2. Idea: Su composición incluye técnicas como la segmentación de audio.
   - Por qué es ambigua: No se aclara qué se entiende por "deteción de silencios" en la segmentación de audio.
   - Párrafo(s): Se encuentra en el segundo párrafo del texto.

3. Idea: Es difícil obtener resultados fiables en diferentes acentos y entornos ruidosos.
   - Por qué es ambigua: No se aclara qué se entiende exactamente por "acentos" y "entornos ruidosos".
   - Párrafo(s): Se encuentra en el segundo párrafo del texto.

4. Idea: El aprendizaje profundo ha propiciado el estudio de enfoques exitosos para entrenar pasos de modelado separados.
   - Por qué es ambigua: No se aclara qué se en

In [ ]:
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted
from dotenv import load_dotenv
from google.generativeai.types import HarmCategory, HarmBlockThreshold

In [ ]:
import re
import os
import time
from pprint import pprint
from fireworks.client import Fireworks
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

API_KEY = os.environ.get("FIREWORKS_API_KEY")
API_MODEL = os.environ.get("FIREWORKS_MODEL")
fw = Fireworks(api_key=API_KEY)  # Fixed variable name (was api_key)


# To use a real LLM, you would install the library, e.g., 'pip install openai'
# from openai import OpenAI

# --- Configuration ---
MAX_PARAGRAPHS_PER_CHUNK = 5
WORD_LIMIT = 400
MODEL_ID = API_MODEL
NUM_ATTEMPTS_PER_CHUNK = 3


class AmbiguityAnalyzer:
    def __init__(self, llm_client, word_limit=WORD_LIMIT, max_paragraphs=MAX_PARAGRAPHS_PER_CHUNK, num_attempts=NUM_ATTEMPTS_PER_CHUNK):
        self.client = llm_client
        self.word_limit = word_limit
        self.max_paragraphs = max_paragraphs
        self.num_attempts = num_attempts
        self.sectioning_pattern = re.compile(r'^\s*\\(chapter|section|subsection|subsubsection)\{([^}]+)\}')
        self.paragraph_pattern = re.compile(r'^\s*\\(chapter|section|subsection|subsubsection)\b')

    # --- Methods for Ambiguity Analysis (Existing Logic) ---

    def _get_paragraphs(self, full_text: str, max_words, paragraph_amount) -> list[str]:
        lines = full_text.splitlines()
        result = []
        curr_chunk = 0
        index = 0
        word_count = 0
        paragraph_count = 0
        while index < len(lines):
            line = lines[index]
            # Check for chapter or section commands
            match = re.match(r'^\s*\\(chapter|part|(sub)*section)\*?\{(.+?)\}', line)
            
            if match:
                command = match.group(1)  # e.g., "section", "chapter", etc.
                title = match.group(3)
                text = f"{command}: {title}"
                if len(result) <= curr_chunk:
                    result.append(f"{text} \n")
                else:
                    result[curr_chunk] +=f"{text} \n"
                word_count += len(text)
            elif line.strip():
                word_count += len(line.split())
                paragraph_count += 1
                if len(result) <= curr_chunk:
                    result.append(f"{line} \n")
                else:
                    result[curr_chunk] += f"{line} \n"
                if word_count >= max_words or paragraph_count >= paragraph_amount:
                    curr_chunk +=1
                    word_count = 0
                    paragraph_count = 0
            index+=1
        return result

                


                # return [line.strip() for line in lines if line.strip() and not self.paragraph_pattern.match(line)]

    def _parse_llm_ambiguity_response(self, response_text: str) -> list[dict]:
        found = []
        pattern = re.compile(r"Idea:\s*\[(.*?)\].*?Por qué es ambigua:\s*(.*?)(?=\n\d+\.\s*Idea:|\Z)", re.DOTALL | re.MULTILINE)
        for match in pattern.finditer(response_text):
            found.append({"ambiguous_sentence": match.group(1).strip(), "reason": match.group(2).strip()})
        return found

    def _deduplicate_results(self, results: list[dict]) -> list[dict]:
        unique, seen = [], set()
        for result in results:
            if result['ambiguous_sentence'] not in seen:
                unique.append(result)
                seen.add(result['ambiguous_sentence'])
        return unique

    def _ask_llm(self, prompt: str) -> str:
        """General-purpose method to query the LLM."""
        try:
            response = self.client.chat.completions.create(model=MODEL_ID, messages=[{"role": "user", "content": prompt}])
            time.sleep(5)
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"An error occurred while calling the LLM API: {e}")
            return ""

    def analyze_ambiguity(self, latex_text: str) -> list[dict]:
        """Main method to process the entire document for ambiguities."""
        print("\n" + "="*20 + " 1. AMBIGUITY ANALYSIS " + "="*20)
        paragraphs = self._get_paragraphs(latex_text, self.word_limit, self.max_paragraphs)
        
        current_idx = 0
        responses = []
        while current_idx < len(paragraphs):
            chunk_text = paragraphs[current_idx]
            print(f"\n--- Analyzing chunk number {current_idx} of paragraphs: {paragraphs[current_idx][:150]}---")
            prompt = self._create_ambiguity_prompt(chunk_text)
            for i in range(self.num_attempts):
                llm_response = self._ask_llm(prompt)
                if llm_response:
                    if len(responses) <= current_idx:
                        responses.append(f"{llm_response} \n")
                    responses[current_idx] += f"{llm_response} \n"
                    i+=1
            current_idx +=1

        return responses

    def _create_ambiguity_prompt(self, text_chunk: str) -> str:
        return (
            "Tienes el siguiente capítulo de una tesis en español. Tu tarea consiste en:\n"
            "1. Identificar las ideas que queden ambiguas o los conceptos que haga falta explicar porque dependan del contexto,  por ejemplo, los adverbios de modo en ocasiones dependen del punto de referencia.\n"
            f"### Texto a analizar:\n{text_chunk}\n\n"
            "### Formato de respuesta (usa este formato estricto):\n"
            "Idea: [frase exacta del texto]\n"
            "   - Por qué es ambigua: [explicación]\n"
            "Si no hay ideas ambiguas devolver: NO"
        )
    
    # --- Methods for Structure and Transition Analysis (New Logic) ---

    def _extract_headings(self, latex_text: str) -> str:
        """Extracts and formats chapter/section headings into a readable list."""
        headings = []
        indent_map = {"chapter": 0, "section": 2, "subsection": 4, "subsubsection": 6}
        for line in latex_text.splitlines():
            match = self.sectioning_pattern.match(line)
            if match:
                level, title = match.groups()
                indent = " " * indent_map.get(level, 0)
                headings.append(f"{indent}- {level.capitalize()}: {title}")
        return "\n".join(headings)

    def analyze_structure(self, latex_text: str) -> str:
        """Analyzes the logical distribution of topics based on headings."""
        print("\n" + "="*20 + " 2. STRUCTURE ANALYSIS " + "="*20)
        headings_list_str = self._extract_headings(latex_text)
        if not headings_list_str:
            return "No section headings found to analyze."
            
        print("--- Sending the following structure to LLM for review: ---")
        print(headings_list_str)
        
        prompt = (
            "Eres un editor académico experto. Analiza el siguiente índice de una tesis. "
            "¿La distribución de temas es lógica? ¿El flujo de un capítulo/sección al siguiente parece coherente? "
            "Proporciona un análisis conciso y sugerencias de mejora si es necesario.\n\n"
            f"### Índice del Documento:\n{headings_list_str}\n\n"
            "### Análisis:"
        )
        return self._ask_llm(prompt)

    def _parse_document_structure(self, latex_text: str) -> list:
        """Parses LaTeX text into a hierarchical list of chapters and sections."""
        lines = latex_text.splitlines()
        document_structure = []
        current_chapter = None
        current_section = None

        for line in lines:
            line = line.strip()
            if not line:
                continue

            chapter_match = re.match(r'^\s*\\(chapter|part)\*?\{', line)
            section_match = re.match(r'^\s*\\(sub)*section\*?\{', line)

            if chapter_match:
                if current_chapter:
                    document_structure.append(current_chapter)
                current_chapter = {
                    "title": chapter_match.group(1),
                    "intro_paragraph": None,
                    "sections": []
                }
                current_section = None # Reset section when a new chapter starts
            elif section_match and current_chapter:
                current_section = {
                    "title": section_match.group(1),
                    "first_paragraph": None,
                    "last_paragraph": None,
                }
                current_chapter["sections"].append(current_section)
            elif not self.paragraph_pattern.match(line): # It's a paragraph
                if current_chapter and not current_section and not current_chapter["intro_paragraph"]:
                    current_chapter["intro_paragraph"] = line
                elif current_section:
                    if not current_section["first_paragraph"]:
                        current_section["first_paragraph"] = line
                    current_section["last_paragraph"] = line # Always update to get the last one
        
        if current_chapter: # Append the last chapter
            document_structure.append(current_chapter)
            
        return document_structure

    def analyze_transitions(self, latex_text: str) -> list[dict]:
        """Analyzes the narrative flow within and between sections for each chapter."""
        print("\n" + "="*20 + " 3. TRANSITION ANALYSIS " + "="*20)
        transitions_text, general_structure = parse_document_structure(latex_text)

        
        prompt = (f"""A continuación tienes un conjunto de inicios de capítulo y secciones. El inicio del capítulo tiene la estructura:
                   
                  y el inicio de una sección tiene la estructura:
                  Tu tarea consiste en:
            1. Determinar si el párrafo con el que comienza el capítulo sirve como pequeña introducción al tema del capítulo.
            2. Por cada sección determinar si el párrafo que la precede y el que la sigue están interconectados y si hay una transición entre ellos. El último párrafo de una sección debería conectar de alguna manera con el primero de la siguiente.
            ### Texto a analizar:{transitions_text}
            ### Formato de respuesta (usa este formato estricto):
            Si es capítulo: Introducción o no. En caso de no explicar cómo se puede mejorar el párrafo.
            Si es sección: Hay transición o no. En caso de no explicar cómo se puede mejorar cada párrafo."""
        )
            
        analysis = self._ask_llm(prompt)

        return analysis


if __name__ == "__main__":
    sample_latex_text = r"""
\chapter{Capítulo 1: Fundamentos para el diseño de un sistema de escritura con emojis.}

Este capítulo presenta los conceptos que permiten abordar el problema del acceso ágil a emojis desde una perspectiva técnica. Se analizan tres ejes fundamentales: el vínculo entre escritura digital y emojis, los principios ergonómicos que optimizan el uso del teclado, y los enfoques semánticos que permiten representar el significado de frases y símbolos en espacios vectoriales.


\section*{Interacción humano-teclado y escritura con emojis}

Más del 90\,\% de los usuarios de internet utilizan emojis de forma habitual, según el \textit{Emoji Trend Report 2021} del Unicode Consortium~\cite{unicode2021emoji}. Su uso mejora la claridad emocional de los mensajes y se ha vuelto esencial en plataformas de mensajería, redes sociales y entornos digitales informales. Por esta razón los emojis se han transformado  en una herramienta expresiva tan significativa como las palabras.

A pesar de esta integración masiva en la comunicación escrita, los métodos actuales para insertar emojis siguen siendo lentos e ineficientes. Navegar por menús gráficos, copiar desde sitios web o memorizar códigos Unicode interrumpe la fluidez de la escritura. Estas barreras afectan especialmente a quienes escriben en teclados físicos, donde no existen accesos rápidos integrados.

El teclado continúa siendo la herramienta principal para escribir en dispositivos digitales. Su diseño, centrado en el alfabeto, no contempla de forma nativa el uso de emojis, lo que agrava el problema. Frente a esta limitación, se propone una solución que mejore la experiencia de escritura con emojis desde dos enfoques: asignar los emojis más usados a teclas físicas de forma ergonómica, y ofrecer un buscador semántico que sugiera emojis a partir del significado de frases escritas.

\section*{Principios de ergonomía aplicados al teclado}

La ergonomía estudia cómo adaptar herramientas y entornos de trabajo a las capacidades físicas del cuerpo humano. Desde el punto de vista de la escritura digital, un diseño ergonómico del teclado busca reducir el esfuerzo físico, evitar tensiones musculares y mejorar la velocidad y precisión al escribir. Estos principios permiten optimizar la distribución de tareas entre los dedos, minimizar desplazamientos innecesarios y facilitar una escritura más cómoda, rápida y sostenida en el tiempo.


La escritura prolongada sin considerar criterios ergonómicos puede provocar incomodidad o fatiga muscular, especialmente en usuarios que utilizan el teclado como herramienta principal. 
\section*{Técnica clásica de mecanografía}

La mecanografía establece una distribución estándar de los dedos sobre el teclado ~\cite{zappala1993touch}, orientada a optimizar la velocidad, precisión y ergonomía durante la escritura. Cada dedo tiene asignada una posición de reposo natural, desde la cual debe moverse de forma controlada para alcanzar las teclas correspondientes a cada uno. Esta distribución considera tanto la fuerza como la movilidad de cada dedo, buscando minimizar desplazamientos innecesarios y reducir la fatiga muscular \cite{norman1990}.

La organización parte de una \textit{fila guía o base} , que es aquella donde descansan los dedos cuando las manos están en posición inicial sobre el teclado. Cada dedo se coloca en una tecla específica sobre esta fila, como se muestra en la Figura~\ref{fig: posicion_reposo}.


\subsection*{Distribución de teclas asignadas a cada dedo}

Cada dedo tiene asignado un conjunto específico de teclas distribuidas en las tres filas principales del teclado alfabético: fila superior, fila guía (base) y fila inferior. Esta distribución sigue el principio de que cada dedo se encargue de un bloque vertical de teclas (columna virtual), minimizando el cruce de dedos y maximizando la eficiencia de la escritura. La Figura \ref{fig: distribución_dedos}  resume esta asignación. 

Los pulgares descansan sobre la barra espaciadora. A diferencia de los demás dedos, no participan en la pulsación de letras, por lo que no se incluyen en la distribución anterior.

Esta técnica busca reducir el esfuerzo físico durante la escritura al asignar teclas cercanas a cada dedo según su movilidad y fuerza. De esta manera, se favorece una mecanografía más rápida~\cite{zappala1993touch,norman1990}.

Este modelo será aplicado en el Capítulo~3 como base para la asignación de emojis, combinando su frecuencia de uso con el costo ergonómico de cada tecla.
La interacción ergonómica, aunque clave para agilizar el acceso a los emojis más usados, no resuelve situaciones donde el usuario no sabe exactamente qué emoji insertar. Para estos casos, es necesario incorporar una lógica de búsqueda más inteligente que relacione el contenido semántico del texto con el significado de los emojis.

\subsection*{Recuperación de información basada en significado}

La recuperación de información semántica busca encontrar contenidos relevantes no por coincidencia literal de palabras, sino por su similitud conceptual. A diferencia de los sistemas clásicos de búsqueda que operan con cadenas de texto exactas, un enfoque semántico permite interpretar el significado subyacente de las frases y establecer relaciones con elementos relacionados, aunque estos no compartan las mismas palabras \cite{manning2008}.

Este paradigma se aplica para sugerir emojis que estén vinculados al sentido general de una frase escrita por el usuario. Por ejemplo, ante la frase “extraño a mi familia”, el sistema debería ser capaz de sugerir emojis como \notaparaelautor{Aquí van unos emojis en cuanto descubra como ponerlos.}, incluso si esas palabras no aparecen literalmente en la base de datos.

Este tipo de recuperación se basa en representar tanto frases como conceptos (en este caso, emojis) en un espacio vectorial común. En ese espacio, elementos similares ocupan posiciones cercanas, lo que permite calcular la relevancia mediante medidas de distancia o similitud \cite{jurafsky2023}.

El diseño del módulo semántico se apoya en estos principios, que serán detallados en los siguientes epígrafes junto a las técnicas específicas de vectorización utilizadas.

\subsection*{Representación vectorial de frases y emojis}

Para realizar una búsqueda semántica eficiente, es necesario transformar tanto las frases escritas por el usuario como los significados asociados a los emojis en una representación matemática comparable. Este proceso se logra mediante la vectorización, que convierte las unidades lingüísticas en vectores dentro de un espacio semántico.

Una técnica ampliamente utilizada para ello es la representación mediante modelos de palabras incrustadas (\textit{word embeddings}), como Word2Vec, GloVe o FastText. Estos modelos aprenden a representar palabras como puntos en un espacio de múltiples dimensiones, de forma que aquellas que aparecen en contextos similares tienden a ubicarse cerca entre sí \cite{mikolov2013efficient}.

Cuando se trabaja con frases completas, se requiere una estrategia de composición semántica. Una opción simple consiste en promediar los vectores de las palabras que componen la frase. Alternativamente, pueden emplearse técnicas más robustas como la Reducción de Dimensionalidad mediante Análisis Semántico Latente (LSA) o métodos de aprendizaje profundo como \textit{Transformers}, aunque estos últimos suelen requerir mayor capacidad computacional \cite{deerwester1990}.

En el caso de los emojis, también se puede construir una representación vectorial utilizando descripciones léxicas asociadas a cada uno (por ejemplo, las ofrecidas por Emojipedia). Al vectorizar esas descripciones con el mismo modelo que se usa para las frases, se garantiza que ambas entidades —texto y emoji— puedan compararse directamente mediante medidas como la similitud del coseno.

Este enfoque permite establecer correspondencias semánticas entre una entrada textual y un conjunto de emojis candidatos, con base en la proximidad geométrica entre sus vectores. Este mecanismo constituye el núcleo del sistema de búsqueda implementado en el Capítulo~3.

\subsection*{Proceso general de búsqueda semántica}

El sistema de búsqueda semántica propuesto sigue una secuencia de etapas que permiten transformar una entrada textual en una sugerencia relevante de emojis. A nivel conceptual, este proceso puede dividirse en cuatro fases principales:


Entrada de texto: el usuario escribe una frase o palabra desde la cual desea obtener sugerencias de emojis.
    
Vectorización: el sistema transforma esa entrada en un vector semántico, utilizando un modelo previamente entrenado para representar el significado del texto.
    
Comparación semántica: se calcula la similitud entre el vector de la entrada y los vectores asociados a los distintos emojis. Estos últimos se obtienen a partir de sus descripciones textuales, procesadas y vectorizadas del mismo modo que la entrada.
    
Sugerencia de emojis: el sistema selecciona aquellos emojis con mayor similitud semántica, los ordena y los presenta al usuario como resultados.


Este mecanismo permite realizar sugerencias incluso cuando no existe coincidencia textual exacta entre lo escrito y la información asociada a los emojis. Así, se mejora la accesibilidad y relevancia de los resultados, especialmente en contextos donde el usuario no recuerda el nombre exacto del emoji deseado.

La implementación de estas etapas, junto con la construcción de la base de datos y las decisiones técnicas específicas, se detalla en el Capítulo~3.
"""
    # Use the mock client for the demo
    fw_llm_client = fw
    analyzer = AmbiguityAnalyzer(llm_client=fw_llm_client)

    # --- Run all three analyses ---

    # # 1. Ambiguity Analysis
    # ambiguity_results = analyzer.analyze_ambiguity(sample_latex_text)
    # print("\n--- Final Ambiguity Results ---")
    # pprint(ambiguity_results)

    # # 2. Structure Analysis
    # structure_analysis = analyzer.analyze_structure(sample_latex_text)
    # print("\n--- Final Structure Analysis ---")
    # print(structure_analysis)

    # # 3. Transition Analysis
    # transition_analysis_results = analyzer.analyze_transitions(sample_latex_text)
    # print("\n--- Final Transition Analysis Results ---")
    # pprint(transition_analysis_results)

IndentationError: unexpected indent (3825833894.py, line 238)

In [ ]:
import re

def get_text_chunks(text, max_words: int, paragraph_amount: int) -> list[str]:
    # text va a ser una lista de tuplas (índice de línea en texto original, línea)
    # lines = full_text.splitlines()
    result = []
    curr_chunk = ""
    word_count = 0
    paragraph_count = 0

    for line in text:

        # Check for LaTeX structural commands
        match = re.match(r'^\s*\\(chapter|part|(sub)*section)\*?\{(.+?)\}', line)
        if match:
            command = match.group(1)
            title = match.group(3)
            line_text = f"{command}: {title} \n"
            word_count += len(title.split())
        else:
            line_text = line
            word_count += len(line.split())
            paragraph_count += 1

        curr_chunk += line_text + " "

        if word_count >= max_words or paragraph_count >= paragraph_amount:
            result.append(curr_chunk)
            # result.append(curr_chunk.strip())
            curr_chunk = ""
            word_count = 0
            paragraph_count = 0

    # Append any remaining text
    if curr_chunk:
        result.append(curr_chunk)

    return result


In [ ]:


# --- Run all three analyses ---
# 1. Ambiguity Analysis
ambiguity_results = get_text_chunks(sample_latex_text, 400, 5)
print("\n--- Final Ambiguity Results ---")
i=0
output = ""
while i < len(ambiguity_results):
    output += "chunk number " + f"{i}" + ambiguity_results[i] + "\n"
    i+=1
with open("output.tex", "w", encoding="utf-8") as f:
        f.write(output)
        f.write("\n")
    


--- Final Ambiguity Results ---


In [15]:
text_chunk = """chapter: Capítulo 1: Fundamentos para el diseño de un sistema de escritura con emojis. 
 Este capítulo presenta los conceptos que permiten abordar el problema del acceso ágil a emojis desde una perspectiva técnica. Se analizan tres ejes fundamentales: el vínculo entre escritura digital y emojis, los principios ergonómicos que optimizan el uso del teclado, y los enfoques semánticos que permiten representar el significado de frases y símbolos en espacios vectoriales. section: Interacción humano-teclado y escritura con emojis 
 Más del 90\,\% de los usuarios de internet utilizan emojis de forma habitual, según el \textit{Emoji Trend Report 2021} del Unicode Consortium~\cite{unicode2021emoji}. Su uso mejora la claridad emocional de los mensajes y se ha vuelto esencial en plataformas de mensajería, redes sociales y entornos digitales informales. Por esta razón los emojis se han transformado  en una herramienta expresiva tan significativa como las palabras. A pesar de esta integración masiva en la comunicación escrita, los métodos actuales para insertar emojis siguen siendo lentos e ineficientes. Navegar por menús gráficos, copiar desde sitios web o memorizar códigos Unicode interrumpe la fluidez de la escritura. Estas barreras afectan especialmente a quienes escriben en teclados físicos, donde no existen accesos rápidos integrados. El teclado continúa siendo la herramienta principal para escribir en dispositivos digitales. Su diseño, centrado en el alfabeto, no contempla de forma nativa el uso de emojis, lo que agrava el problema. Frente a esta limitación, se propone una solución que mejore la experiencia de escritura con emojis desde dos enfoques: asignar los emojis más usados a teclas físicas de forma ergonómica, y ofrecer un buscador semántico que sugiera emojis a partir del significado de frases escritas. section: Principios de ergonomía aplicados al teclado 
 La ergonomía estudia cómo adaptar herramientas y entornos de trabajo a las capacidades físicas del cuerpo humano. Desde el punto de vista de la escritura digital, un diseño ergonómico del teclado busca reducir el esfuerzo físico, evitar tensiones musculares y mejorar la velocidad y precisión al escribir. Estos principios permiten optimizar la distribución de tareas entre los dedos, minimizar desplazamientos innecesarios y facilitar una escritura más cómoda, rápida y sostenida en el tiempo.
"""
prompt = f"""Tienes el siguiente capítulo de una tesis en español. Tu tarea consiste en:
            1. Identificar las ideas que queden ambiguas o los conceptos que haga falta explicar porque dependan del contexto,  por ejemplo, los adverbios de modo en ocasiones dependen del punto de referencia.
            ### Texto a analizar:{text_chunk}
            ### Formato de respuesta (usa este formato estricto):
            Idea: [frase exacta del texto]
               - Por qué es ambigua: [explicación]
            Si no hay ideas ambiguas devolver: NO"""
llm_response = analyzer._ask_llm(prompt)
print(llm_response)

<>:1: SyntaxWarning: invalid escape sequence '\,'
<>:1: SyntaxWarning: invalid escape sequence '\,'
C:\Users\Melissa\AppData\Local\Temp\ipykernel_9876\430798970.py:1: SyntaxWarning: invalid escape sequence '\,'
  text_chunk = """chapter: Capítulo 1: Fundamentos para el diseño de un sistema de escritura con emojis.


Aquí te presento las ideas ambiguas que he identificado en el texto:

1. Idea: "Más del 90\,\% de los usuarios de internet utilizan emojis de forma habitual, según el 	extit{Emoji Trend Report 2021} del Unicode Consortium~\cite{unicode2021emoji}."
   - Por qué es ambigua: No se especifica qué se entiende por "utilizar emojis de forma habitual". ¿Se refiere a utilizar emojis en cada mensaje, en la mayoría de los mensajes, o simplemente a que los usuarios conocen y están familiarizados con ellos?

2. Idea: "Su uso mejora la claridad emocional de los mensajes y se ha vuelto esencial en plataformas de mensajería, redes sociales y entornos digitales informales."
   - Por qué es ambigua: No se define qué se entiende por "claridad emocional de los mensajes". ¿Se refiere a la capacidad de expresar sentimientos y emociones de manera efectiva, o a la capacidad de comunicar la intención o el tono de un mensaje de manera clara?

3. Idea: "Navegar por menús gráficos, copiar desde sitios web o memor

In [ ]:
melissa_text = """
\chapter{Propuesta}\label{chapter:proposal}

Dado que no se dispone de un corpus en español anotado específicamente con \textbf{frases de comadreja} —como se explicó en los preliminares—, este programa opta por utilizar una lista previamente compilada de palabras y expresiones que contribuyen a que el texto se perciba como impreciso y evasivo. Para construir dicha lista, se consultaron varios repositorios disponibles en GitHub \cite{} que se dedicaban a identificar \textbf{frases de comadreja} mediante listas predefinidas.

En los casos en que las listas estaban en inglés, las entradas se tradujeron manualmente. Si, en cambio, estaban en español, se incorporaron directamente a la lista. Además, se recurrió a la colaboración de profesores de la facultad, quienes aportaron ejemplos reales de frases detectadas durante la corrección de tesis académicas.

Una vez reunidos todos los términos, se utilizó un script en Python para eliminar duplicados, garantizando que cada elemento aparezca solo una vez en la lista final. Esta lista fue luego revisada manualmente para descartar expresiones que se pueden utilizar sin generar imprecisión en el texto, por lo cual sería incorrecto clasificarlas como comadrejas.

\section{voz pasiva}
Además de las frases de comadreja, otro tipo de error que este trabajo detecta es el uso de la \textbf{voz pasiva}. Para identificar estas construcciones, se implementó un método que aprovecha las capacidades de análisis gramatical que ofrece \textit{spaCy}. Esta herramienta permite examinar cada palabra del texto y asignarle dos propiedades fundamentales: su \textit{lema} —es decir, su forma base o canónica, como ``ser'' en lugar de ``es'' o ``fue''— y su \textit{categoría gramatical}, también conocida como parte del discurso o \textit{part of speech} (POS), que indica si se trata de un verbo, sustantivo, adjetivo, entre otros. Luego se recorre el texto palabra por palabra, buscando patrones característicos de la voz pasiva en español.

Se considera un patrón de voz pasiva a una palabra cuya categoría gramatical es verbo o auxiliar y su lema es ``ser'', seguida de un participio. Sin embargo, \textit{spaCy} no es perfecto por lo que no identifica todos los participios correctamente. Para compensar esto se comprueba que la palabra a continuación de la forma verbal de ``ser'' termine en ``ado'', ``ido'', ``to'', ``so'', ``cho'' y no sea un adjetivo, porque lo que es válido que a continuación de la forma verbal de ser venga un adjetivo y algunos de ellos terminan igual que los participios, pero en cualquier otro caso se considera que lo que viene es un participio, aunque \textit{spaCy} no lo reconozca como tal. 

\section{3ra persona}
Otro aspecto que se analiza en este trabajo es la presencia de \textbf{expresiones en primera o segunda persona gramatical}. Para su detección, se desarrolló un método que también utiliza \textit{spaCy}. En este caso se busca señalar tanto los pronombres personales como las formas verbales correspondientes a la primera y segunda persona del español.

Para la detección de pronombres personales en primera y segunda persona, se definió una lista explícita que incluye formas como yo, me, tú, nosotros, entre otras. Aunque \textit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo. Dado que el conjunto de pronombres personales en español es limitado y bien definido, se optó por no delegar esta tarea a \textit{spaCy}  y utilizar directamente una lista controlada. Durante el análisis, si alguno de los elementos de esa lista aparece en el texto y ha sido etiquetado como pronombre (es decir, su categoría gramatical es PRON), se registra su posición para poder marcarlo posteriormente.

Por otro lado, también se revisa si los verbos del texto —ya sean simples o compuestos— están conjugados en primera o segunda persona. Para ello, se consultan los rasgos morfológicos de cada verbo, en particular el atributo Person, que indica si se trata de una forma correspondiente a la persona 1 o 2.

En el caso de los verbos compuestos, si se detecta un verbo auxiliar en primera o segunda persona (por ejemplo, he, has, hemos), el algoritmo examina los tokens siguientes para verificar si están acompañados de un participio como escrito, leído o visto. Si se encuentra esta estructura, se registra toda la expresión como una construcción en primera o segunda persona, por ejemplo: he escrito o has visto.

\section{anglicismos}
También se elaboró una lista de anglicismos con el objetivo de detectarlos en el texto. Para su confección, se recopilaron términos a partir de la fuente \cite{} y mediante consultas a profesores de la facultad, quienes identificaron los anglicismos más frecuentes en trabajos académicos.

Para señalar todos estos errores, se comenta el fragmento de texto correspondiente con un comando de \LaTeX, como se muestra en la siguiente imagen:


Las repeticiones innecesarias de palabras dentro de una oración pueden dificultar la lectura, del mismo modo que las oraciones excesivamente largas pueden generar confusión y dificultar la comprensión del mensaje. Para identificar estos aspectos, el sistema permite al usuario definir dos parámetros: la cantidad máxima de palabras a partir de la cual una oración se considera demasiado larga, y el rango (también en número de palabras) dentro del cual una palabra repetida se considera problemática. Adicionalmente, se establece que una palabra se considera repetida si aparece al menos tres veces dentro de un mismo párrafo. Las oraciones largas se señalan mediante un comando específico, mientras que las palabras repetidas se marcan con un mismo color y se les asigna un índice que permite identificar todas las apariciones de una misma palabra dentro del párrafo. A continuación, se presenta un ejemplo ilustrativo que muestra cómo se señalan estos elementos en el texto:"""

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Melissa\AppData\Local\Temp\ipykernel_9876\3818525153.py:1: SyntaxWarning: invalid escape sequence '\c'
  melissa_text = """


In [ ]:
ambiguity_results = get_text_chunks(melissa_text, 400, 5)
print("\n--- Final Ambiguity Results ---")
i=0
output = ""
while i < len(ambiguity_results):
    output += "chunk number " + f"{i}" + ambiguity_results[i] + "\n"
    i+=1
with open("output_MELISSA.tex", "w", encoding="utf-8") as f:
        f.write(output)
        f.write("\n")


--- Final Ambiguity Results ---


In [30]:
text_chunk = """Otro aspecto que se analiza en este trabajo es la presencia de 	extbf{expresiones en primera o segunda persona gramatical}. Para su detección, se desarrolló un método que también utiliza 	extit{spaCy}. En este caso se busca señalar tanto los pronombres personales como las formas verbales correspondientes a la primera y segunda persona del español. Para la detección de pronombres personales en primera y segunda persona, se definió una lista explícita que incluye formas como yo, me, tú, nosotros, entre otras. Aunque 	extit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo. Dado que el conjunto de pronombres personales en español es limitado y bien definido, se optó por no delegar esta tarea a 	extit{spaCy}  y utilizar directamente una lista controlada. Durante el análisis, si alguno de los elementos de esa lista aparece en el texto y ha sido etiquetado como pronombre (es decir, su categoría gramatical es PRON), se registra su posición para poder marcarlo posteriormente. Por otro lado, también se revisa si los verbos del texto —ya sean simples o compuestos— están conjugados en primera o segunda persona. Para ello, se consultan los rasgos morfológicos de cada verbo, en particular el atributo Person, que indica si se trata de una forma correspondiente a la persona 1 o 2. En el caso de los verbos compuestos, si se detecta un verbo auxiliar en primera o segunda persona (por ejemplo, he, has, hemos), el algoritmo examina los tokens siguientes para verificar si están acompañados de un participio como escrito, leído o visto. Si se encuentra esta estructura, se registra toda la expresión como una construcción en primera o segunda persona, por ejemplo: he escrito o has visto. También se elaboró una lista de anglicismos con el objetivo de detectarlos en el texto. Para su confección, se recopilaron términos a partir de la fuente \cite{} y mediante consultas a profesores de la facultad, quienes identificaron los anglicismos más frecuentes en trabajos académicos.
"""
prompt = f"""Tienes el siguiente capítulo de una tesis en español. Tu tarea consiste en:
            1. Identificar las ideas que queden ambiguas o los conceptos que haga falta explicar porque dependan del contexto,  por ejemplo, los adverbios de modo en ocasiones dependen del punto de referencia. Si la idea está explicada en alguna parte del texto entonces no se considera ambigua.
            ### Texto a analizar:{text_chunk}
            ### Formato de respuesta (usa este formato estricto):
            Si no hay ideas ambiguas devolver: NO
            Si hay ideas ambiguas devolver:
            Idea: [frase exacta del texto]
               - Por qué es ambigua: [explicación]
            """
llm_response = analyzer._ask_llm(prompt)
print(llm_response)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Melissa\AppData\Local\Temp\ipykernel_9876\1560139920.py:1: SyntaxWarning: invalid escape sequence '\c'
  text_chunk = """Otro aspecto que se analiza en este trabajo es la presencia de 	extbf{expresiones en primera o segunda persona gramatical}. Para su detección, se desarrolló un método que también utiliza 	extit{spaCy}. En este caso se busca señalar tanto los pronombres personales como las formas verbales correspondientes a la primera y segunda persona del español. Para la detección de pronombres personales en primera y segunda persona, se definió una lista explícita que incluye formas como yo, me, tú, nosotros, entre otras. Aunque 	extit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo. Dado que el conjunto de pronombres personales en español es limitado y bien definido, se optó p

1.  Idea: "Aunque 	extit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo."
    -   Por qué es ambigua: Esta idea es ambigua porque no se especifica claramente qué se entiende por "errores u omisiones". ¿Se refiere a que  spaCy puede identificar incorrectamente la persona gramatical de los pronombres, o que puede omitir pronombres personales en la detección?

2.  Idea: "Por ejemplo: he escrito o has visto."
    -   Por qué es ambigua: Esta idea es ambigua porque no se especifica claramente qué se entiende por "estructura". ¿Se refiere a la estructura gramatical del verbo auxiliar seguido de un participio, o a una estructura sintáctica más amplia?

3.  Idea: "Para su confección, se recopilaron términos a partir de la fuente \cite{} y mediante consultas a profesores de la facultad, quienes identificaron los anglicismos más frecuentes en trabajos académicos."
    -   Por qué es ambigua: Esta 

In [31]:
prompt = f"""Tienes el siguiente capítulo de una tesis en español y una lista de ideas. Tu tarea consiste en:
            1. Identificar si las ideas quedan ambiguas, es decir, que leyendo el capítulo no es posible aclarar las preguntas que te haces al leerlas.
            ### Texto a analizar:{text_chunk}
            ### Ideas: {llm_response}
            ### Formato de respuesta (usa este formato estricto):
            Idea: [idea]
            Ambigua/No ambigua: [Si/No]
            Por qué: []
            """
new_response = analyzer._ask_llm(prompt)
print(new_response)

Idea: "Aunque 	extit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo."
Ambigua/No ambigua: Ambigua
Por qué: No se especifica claramente qué se entiende por "errores u omisiones". ¿Se refiere a que  spaCy puede identificar incorrectamente la persona gramatical de los pronombres, o que puede omitir pronombres personales en la detección?

Idea: "Por ejemplo: he escrito o has visto."
Ambigua/No ambigua: Ambigua
Por qué: No se especifica claramente qué se entiende por "estructura". ¿Se refiere a la estructura gramatical del verbo auxiliar seguido de un participio, o a una estructura sintáctica más amplia?

Idea: "Para su confección, se recopilaron términos a partir de la fuente \cite{} y mediante consultas a profesores de la facultad, quienes identificaron los anglicismos más frecuentes en trabajos académicos."
Ambigua/No ambigua: Ambigua
Por qué: No se especifica claramente qué se entiende po

Transitions

In [ ]:
def remove_empty_lines(text: str) -> str:
    lines = text.splitlines()
    non_empty_lines = [line for line in lines if line.strip()]
    return "\n".join(non_empty_lines)


def parse_document_structure(latex_text: str) -> list:
    """Parses LaTeX text into a hierarchical list of chapters and sections."""
    latex_text = remove_empty_lines(latex_text)
    lines = latex_text.splitlines()
    result = ""
    general_structure = []
    i = 0
    while i < len(lines):
        line = lines[i].strip()

        chapter_match = re.match(r'^\s*\\(chapter|part)\*?\{(.+?)\}', line)
        section_match = re.match(r'^\s*\\(sub)*section\*?\{(.+?)\}', line)

        if chapter_match:
            
            title = chapter_match.group(2)  
            general_structure.append(f"Capítulo: {title}")
            result += "{"+ f"Capítulo {title}: "
            i+=1
            line = lines[i]
            if re.match(r'^\s*\\(chapter|part)\*?\{', line) or re.match(r'^\s*\\(sub)*section\*?\{', line):
                result += "no text}\n"
            else:
                result += f"{line}" + "}\n"
                i+=1
            
        elif section_match:
            title = section_match.group(2)
            general_structure.append(f"Sección: {title}")
            result += "{"+ f"Sección {title}: " "Párrafo anterior: {"
            prev_line = lines[i-1]
            if re.match(r'^\s*\\(chapter|part)\*?\{', prev_line) or re.match(r'^\s*\\(sub)*section\*?\{', prev_line):
                result += "no text} Párrafo siguiente a sección: {"
            else:
                result += f"{prev_line}" + "} Párrafo siguiente a sección: {"
            i+=1
            line = lines[i]
            if re.match(r'^\s*\\(chapter|part)\*?\{', line) or re.match(r'^\s*\\(sub)*section\*?\{', line):
                result += "no text}\n"
            else:
                result += f"{line}" + "}\n"
                i+=1
        else:
            i+=1
        
    return result, general_structure


def transition_analyzer(llm_client, latex_text: str) -> list[dict]:
    """Analyzes the narrative flow within and between sections for each chapter."""
    print("\n" + "="*20 + " 3. TRANSITION ANALYSIS " + "="*20)
    transitions_text, general_structure = parse_document_structure(latex_text)

    
    prompt = (
        f"""A continuación tienes un conjunto de inicios de capítulo y secciones. 
    El inicio del capítulo tiene la estructura:
    {{Capítulo nombre_capítulo: 1er párrafo}}

    Y el inicio de una sección tiene la estructura: 
    Sección nombre_de_sección: Párrafo anterior: {{}} Párrafo siguiente a sección: {{}}

    Tu tarea consiste en:

    1. Determinar si el párrafo con el que comienza el capítulo sirve como pequeña introducción al tema del capítulo.

    2. Por cada sección, determinar si el párrafo que la precede y el que la sigue están interconectados y si hay una transición entre ellos. 
    Es decir, si hay una transición de una sección a la otra. 
    El último párrafo de una sección debería conectar de alguna manera con el primero de la siguiente.

    ### Texto a analizar:
    {transitions_text}

    ### Formato de respuesta (usa este formato estricto):

    Para capítulo:
    Introducción: [Sí/No]
    Explicar cómo se puede mejorar el párrafo si la anterior es No: []

    Para sección:
    Transición: [Sí/No]
    Explicar cómo se puede mejorar cada párrafo si la anterior es No: []
    """
    )
    answer = llm_client.ask_llm(prompt)
    answer = parse_transitions_answer(answer)
    return answer, general_structure

def logical_order_analyzer(llm_client, gen_structure):
    structure = "\n".join(gen_structure)

    prompt = (
        f"""A continuación tienes un conjunto de capítulo y secciones que representa el orden en que se trata un tema en un capítulo de una tesis. Si el primer elemento es un capítulo este va a contener el nombre del capítulo a analizar, que siempre viene primero que las secciones y sobre él no debes sugerir cambios.
    Tu tarea consiste en:

    1. Determinar si el orden propuesto para las secciones dado el nombre del capítulo está bien y en caso contrario sugerir otro orden con la justificación de tu propuesta.
    ### Estructura del capítulo:
    {structure}

    ### Formato de respuesta (usa este formato estricto):

    Orden lógico: [Sí/No]
    Explicar cómo se puede mejorar el orden si la anterior es No: []

    """
    )
        
    analysis = llm_client.ask_llm(prompt)
    answer = parse_logical_order_analysis(analysis)

    return analysis

fw_llm_client = fw
analyzer = AmbiguityAnalyzer(llm_client=fw_llm_client)  
a, general_structure = transition_analyzer(analyzer, melissa_text)

print(a)



==================== 3. TRANSITION ANALYSIS ====================
### Para capítulo:

Introducción: Sí

### Para secciones:

#### Sección voz pasiva:

Transición: Sí

#### Sección 3ra persona:

Transición: Sí

#### Sección anglicismos:

Transición: Sí


In [ ]:
b = logical_order_analyzer(general_structure)
print(b)

### Orden lógico: No
### Explicar cómo se puede mejorar el orden si la anterior es No: 
Parece que el orden no es lógico porque la voz pasiva y la tercera persona no tienen relación directa con el propósito principal del capítulo, que es la propuesta. 

Una posible mejora sería mover las secciones de la voz pasiva y la tercera persona al final, ya que probablemente se refieran a aspectos específicos de la propuesta y no al propósito principal del capítulo. 

Aquí te dejo una posible reestructuración:

### Estructura del capítulo:
['Capítulo: Propuesta', 'Sección: anglicismos', 'Sección: voz pasiva', 'Sección: 3ra persona']

En esta reestructuración, la sección de anglicismos es la primera en aparecer porque es probable que se trate de un aspecto general de la propuesta, seguido de la sección de voz pasiva y tercera persona, que es más específica y se refiere a aspectos de la propuesta.


In [ ]:
def parse_ambiguity_response():
    """Este método devuelve una lista con tuplas de la forma: [Idea ambigua, razón] """
    # Formato de LLM response
    # Idea: "Aunque 	extit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo."
    # Ambigua/No ambigua: Ambigua
    # Por qué: No se especifica claramente qué se entiende por "errores u omisiones". ¿Se refiere a que  spaCy puede identificar incorrectamente la persona gramatical de los pronombres, o que puede omitir pronombres personales en la detección?

    # Idea: "Por ejemplo: he escrito o has visto."

    return [["hola", "soy ambigua"]]

In [ ]:


import os
from fireworks.client import Fireworks
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

API_KEY = os.environ.get("FIREWORKS_API_KEY")
API_MODEL = os.environ.get("FIREWORKS_MODEL")
fw = Fireworks(api_key=API_KEY)  # Fixed variable name (was api_key)
model_id = API_MODEL
# fw_llm_client = Fireworks_Api(fw, model_id)

class Fireworks_Api:
    def __init__(self, llm_client, model_id):
        self.client = llm_client
        self.model_id = model_id

    def ask_llm(self, prompt: str) -> str:
        """General-purpose method to query the LLM."""
        try:
            response = self.client.chat.completions.create(model=self.model_id, messages=[{"role": "user", "content": prompt}])
            time.sleep(5)
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"An error occurred while calling the LLM API: {e}")
            return ""
        
def check_ambiguity_and_transitions(text_lines_for_LLM, llm_client, line_mapper):
    chunks = get_text_chunks(text_lines_for_LLM, 400, 5)
    # chunks es un dict en el que las llaves corresponden a la
    mark_ambiguity = {}
    i = 0
    while i < len(chunks):
        text_chunk = chunks[i]
        first_prompt = f"""Tienes el siguiente capítulo de una tesis en español. Tu tarea consiste en:
                1. Identificar las ideas que queden ambiguas o los conceptos que haga falta explicar porque dependan del contexto,  por ejemplo, los adverbios de modo en ocasiones dependen del punto de referencia. Si la idea está explicada en alguna parte del texto entonces no se considera ambigua.
                ### Texto a analizar:{text_chunk}
                ### Formato de respuesta (usa este formato estricto):
                Si no hay ideas ambiguas devolver: NO
                Si hay ideas ambiguas devolver:
                Idea: [frase exacta del texto]
                - Por qué es ambigua: [explicación]
                """
        llm_response = llm_client.ask_llm(first_prompt)

        second_prompt = f"""Tienes el siguiente capítulo de una tesis en español y una lista de ideas. Tu tarea consiste en:
                1. Identificar si las ideas quedan ambiguas, es decir, que leyendo el capítulo no es posible aclarar las preguntas que te haces al leerlas.
                ### Texto a analizar:{text_chunk}
                ### Ideas: {llm_response}
                ### Formato de respuesta (usa este formato estricto):
                Idea: [idea]
                Ambigua/No ambigua: [Si/No]
                Por qué: []
                """
        ambiguous_ideas = parse_ambiguity_response()
        
        for idea in ambiguous_ideas:
            line_idx, sentence_idx = locate_sentence(idea, text_chunk)
            og_line_idx = line_mapper[line_idx]
            if [og_line_idx, sentence_idx] in mark_ambiguity:
                mark_ambiguity[og_line_idx,sentence_idx].append(idea.reason)
            else:
                mark_ambiguity[og_line_idx,sentence_idx] = idea.reason
        i+=1
    
    #transitions
    # vamos a marcar transiciones con nota para el autor
    transitions_answer, general_structure = transition_analyzer(llm_client, "\n".join(text_lines_for_LLM)) # devuelve lista de tuplas de la forma: [sección sin transición, sugerencia]
    logical_order_answer = logical_order_analyzer(llm_client, general_structure) # devuelve None si no hay sugerencias para el orden, en caso contrario devuelve una nota para el inicio del capítulo
    return mark_ambiguity, transitions_answer, logical_order_answer

    
    

IndentationError: unindent does not match any outer indentation level (<string>, line 28)

In [ ]:
import re

def extract_ambiguous_ideas(text: str) -> list[dict]:
    pattern = re.compile(
        r"(?:^\s*#*\s*)?Idea:\s*\"(.+?)\"\s*"                      # optional ### and spaces before Idea:
        r"Ambigua(?:/No ambigua)?:\s*(Sí|No|Ambigua|No ambigua)\s*"
        r"Por qué:\s*(.+?)(?=\n(?:\s*#*\s*)?Idea:|\Z)",            # lookahead for next Idea: possibly prefixed by ### or whitespace
        re.DOTALL | re.MULTILINE
    )
    
    matches = pattern.findall(text)
    result = []
    for idea, status, reason in matches:
        if status.strip().lower() == "ambigua" or status.strip().lower() == "si" or status.strip().lower() == "sí":
            result.append({
                "idea": idea.strip(),
                "reason": reason.strip()
            })
    return result



In [ ]:
text = '''
Idea: "Aunque 	extit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo."
Ambigua/No ambigua: Ambigua
Por qué: No se especifica claramente qué se entiende por "errores u omisiones". ¿Se refiere a que  spaCy puede identificar incorrectamente la persona gramatical de los pronombres, o que puede omitir pronombres personales en la detección?

Idea: "su patron característico"
Ambigua: Sí
Por qué: El término "patrón característico" no está definido explícitamente en el texto, lo que puede llevar a confusiones sobre qué se entiende por "patrón característico" en este contexto. Es posible que se refiera a un patrón lingüístico o gramatical específico, pero no está claro.

Idea: "una lista"
Ambigua: Si
Por qué: No se especifica qué tipo de lista se refiere, si es una lista de palabras, un catálogo de errores, etc.
'''

parsed = extract_ambiguous_ideas(text)
for item in parsed:
    print(item)


{'idea': 'Aunque \textit{spaCy} cuenta con mecanismos para identificar la persona gramatical de los pronombres, en la práctica puede cometer errores u omisiones al hacerlo.', 'reason': 'No se especifica claramente qué se entiende por "errores u omisiones". ¿Se refiere a que  spaCy puede identificar incorrectamente la persona gramatical de los pronombres, o que puede omitir pronombres personales en la detección?'}
{'idea': 'su patron característico', 'reason': 'El término "patrón característico" no está definido explícitamente en el texto, lo que puede llevar a confusiones sobre qué se entiende por "patrón característico" en este contexto. Es posible que se refiera a un patrón lingüístico o gramatical específico, pero no está claro.'}


In [15]:
import re

def extract_sections_without_transition(text: str) -> dict[str, str]:
    result = {}

    # Match blocks starting with 'Nombre de sección:' and capture the block content
    block_pattern = re.compile(
        r"Nombre de sección:\s*(.+?)\s*"
        r"Transición:\s*(Sí|No)\s*"
        r"Sugerencia para incluir transición:\s*(.+?)(?=\nNombre de sección:|\Z)", 
        re.DOTALL
    )

    for name, transition, suggestion in block_pattern.findall(text):
        if transition.strip().lower() == "no":
            result[name.strip()] = suggestion.strip()

    return result




In [17]:
text = """
Nombre de sección: voz pasiva
Transición: Sí
Sugerencia para incluir transición: []

Nombre de sección: anglicismos
Transición: No
Sugerencia para incluir transición: Se puede mejorar enlazando con el contexto anterior.

Nombre de sección: 3ra persona
Transición: No
Sugerencia para incluir transición: Se recomienda hacer una referencia más clara a la sección anterior.
"""

result = extract_sections_without_transition(text)
for key in result.keys():
    print(f"{key}: {result[key]}")


anglicismos: Se puede mejorar enlazando con el contexto anterior.
3ra persona: Se recomienda hacer una referencia más clara a la sección anterior.


In [20]:
import re

def get_intro_suggestion(text: str) -> str | None:
    intro_match = re.search(r'Introducción:\s*(Sí|No)', text, re.IGNORECASE)
    suggestion_match = re.search(r'Sugerencia para mejorar introducción:\s*(.*)', text, re.IGNORECASE)

    if not intro_match or not suggestion_match:
        return None  # missing either part

    if intro_match.group(1).strip().lower() == "sí" or intro_match.group(1).strip().lower() == "si":
        return None

    return suggestion_match.group(1).strip()


In [21]:
text1 = """
Introducción: Sí
Sugerencia para mejorar introducción: Añadir una frase que resuma el contenido del capítulo.
"""

text2 = """
Introducción: No
Sugerencia para mejorar introducción: Agregar una introducción contextual.
"""

print(get_intro_suggestion(text1))  # ✅ Returns: Añadir una frase que resuma el contenido del capítulo.
print(get_intro_suggestion(text2))  # ✅ Returns: None


None
Agregar una introducción contextual.


In [37]:
import re

def get_order_suggestion(text: str) -> str | None:
    order_match = re.search(r'Orden lógico:\s*(Sí|No)', text, re.IGNORECASE)

    if not order_match:
        return None

    if order_match.group(1).strip().lower() in ["sí", "si"]:
        return None

    # Find the position after the label
    label_pattern = re.search(r'Sugerencias de mejora para el orden:\s*', text, re.IGNORECASE)
    
    if not label_pattern:
        return None

    # Get the end position of the match
    start_pos = label_pattern.end()

    # Return everything after that position
    return text[start_pos:].strip()


In [38]:
text1 = """
Orden lógico: Sí
Sugerencias de mejora para el orden: Reordenar los párrafos para empezar con lo más general.
"""

text2 = """
Orden lógico: No
Sugerencias de mejora para el orden: Se recomienda organizar los ejemplos de manera progresiva.
Esto es un ejemplo:
a, b,c
"""

print(get_order_suggestion(text1))  # ✅ Returns: Reordenar los párrafos para empezar con lo más general.
print(get_order_suggestion(text2))  # ✅ Returns: None


None
Se recomienda organizar los ejemplos de manera progresiva.
Esto es un ejemplo:
a, b,c


In [26]:
import spacy

# Load Spanish model (or replace with "en_core_web_sm" for English)
nlp = spacy.load("es_core_news_sm")

def find_fragment_location(text: str, fragment: str) -> tuple[int, int] | None:
    lines = text.splitlines()

    for line_num, line in enumerate(lines):
        doc = nlp(line)
        for sent_num, sent in enumerate(doc.sents):
            if fragment in sent.text:
                return (line_num, sent_num) 
    return None


In [27]:
text = """Este es el primer párrafo.
Aquí comienza una nueva línea. Este fragmento es relevante.
Otra oración que no contiene nada importante."""

fragment = "comienza una"

print(find_fragment_location(text, fragment))  # Output: (2, 2)


(1, 0)


In [2]:
import re

def extract_sections_without_transition(text: str) -> dict[str, str]:
    result = {}

    # Patrón flexible para detectar bloques de sección
    block_pattern = re.compile(
        r"(?:\*\*.*?\*\*\s*)?"  # opción para encabezado con negritas (**Sección voz pasiva**)
        r"Nombre de secci[oó]n:\s*(.+?)\s*"
        r"Transici[oó]n:\s*(Sí|No)\s*"
        r"Sugerencia para incluir transici[oó]n:\s*(.+?)(?=\n(?:\*\*|Nombre de secci[oó]n:|\Z))",
        re.DOTALL | re.IGNORECASE
    )

    for name, transition, suggestion in block_pattern.findall(text):
        if transition.strip().lower() == "no":
            result[name.strip()] = suggestion.strip()

    return result


In [3]:
text = """
Claroo! A continuacion, se presentan las respuestas con el formato solicitado. 

**Seccion palabras de comadreja: voz pasiva** 
Nombre de seccion: palabras de comadreja 
Transicion: No 
Sugerencia para incluir transicion: La seccion de palabras de comadreja deberia explicar cómo estas palabras se encuentran en el texto, para así conectar con la siguiente sección. 

**Sección voz pasiva: final** 
Nombre de sección: voz pasiva 
Transición: Sí 
Sugerencia para incluir transición: No hay necesidad de 
"""

print(extract_sections_without_transition(text))


{'palabras de comadreja': 'La seccion de palabras de comadreja deberia explicar cómo estas palabras se encuentran en el texto, para así conectar con la siguiente sección.'}


LINE MAPPER
0: value: 5
1: value: 12
2: value: 13
3: value: 15
4: value: 16
5: value: 18
6: value: 20
count is: 21
Not found:Patr�n caracter�stico" de la voz pasiva
Ambigua/No ambigua: Ambigua
Por qu�: No se explica qu� se entiende por "patr�n caracter�stico" de la voz pasiva, lo que puede llevar a interpretaciones diferentes.

Idea: "Forma verbal del verbo ser
Not found:Participio" (definici�n)
Ambigua/No ambigua: Ambigua
Por qu�: No se explica qu� se entiende por "participio", lo que puede llevar a confusiones sobre la forma verbal en cuesti�n.

Idea: "Identificar todos los errores

==================== 3. TRANSITION ANALYSIS ====================
### Secci�n 1
Nombre de secci�n: palabras de comadreja
Transici�n: [No]
Sugerencia para incluir transici�n: [Hacer referencia al objetivo de la investigaci�n para conectar con la siguiente secci�n]

### Secci�n 2
Nombre de secci�n: voz pasiva
Transici�n: [S�]
Sugerencia para incluir transici�n: [Se refiere a la aplicaci�n del conocimiento de la voz pasiva en el texto, lo que conecta con la secci�n anterior.]

### Secci�n 3
Nombre de secci�n: final
Transici�n: [S�]
Sugerencia para incluir transici�n: [La secci�n final resume los resultados de la investigaci�n, lo que conecta con el objetivo de la investigaci�n presentado en la primera secci�n.]

{'palabras de comadreja': '[Hacer referencia al objetivo de la investigaci�n para conectar con la siguiente secci�n]'}
Missing sentence at index 1 in line 12
Modified file saved as: a.tex

[Done] exited with code=0 in 33.324 seconds